In [1]:
try:
    !rm -fr TP_3_DL
    !git clone --quiet https://github.com/Aurel37/TP_3_DL.git

except ImportError:
    pass

In [2]:
# Connect colab with Drive
persistent_storage = 'trainings/'
try:
    # Load the Drive helper and mount
    from google.colab import drive
    import os

    # This will prompt for authorization.
    drive.mount( 'Drive')
    persistent_storage = 'Drive/My Drive/MVA/DL'
    os.makedirs(persistent_storage, exist_ok=True)
except:
    pass

Mounted at Drive


In [3]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2022-02-14 15:00:40--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   182MB/s    in 0.4s    

2022-02-14 15:00:41 (182 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h

In [4]:
!conda install -y pytorch torchvision cudatoolkit=10.2 -c pytorch
!conda install -y matplotlib scikit-learn jupyter requests dgl-cuda10.2 -c dglteam

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=10.2
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           6 KB
    bzip2-1.0.8                |       h7b6447c_0          78 KB
    ca-certificates-2021.10.26 |       h06a4308_2         115 KB
    certifi-2021.10.8          |   py37h06a4308_2         151 KB
    conda-4.11.0               |   py37h06a4308_0        14.4 MB
    cudatoolkit-10.2.89        |       hfd86e86_1       365.1 MB
    ffmpeg-4.3                 |       hf484d3e_0         9.9 MB  pytorch
    freetype-2.11.0            |       h70c0345_0         618 KB
    giflib-5.2.1               |       h7b6447c_0          78 KB
    gmp-6.2.1        

In [ ]:
! python /content/TP_3_DL/train_ppi_baseline.py --gpu 0 --epochs 10000

[15:50:58] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /usr/local/lib/python3.7/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.2.so: cannot open shared object file: No such file or directory
Using backend: pytorch
/usr/local/lib/python3.7/site-packages/dgl/data/utils.py:286: UserWarning: Property dataset.features will be deprecated, please use dataset.graphs[i].ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/usr/local/lib/python3.7/site-packages/dgl/data/utils.py:286: UserWarning: Property dataset.labels will be deprecated, please use dataset.graphs[i].ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
Epoch 00001 | Loss: 176.6399
F1-Score: 0.5074 
Epoch 00002 | Loss: 175.5514
Epoch 00003 | Loss: 175.2411
Epoch 00004 | Loss: 174.7514
Epoch 00005 | Loss: 174.3215
Epoch 00006 | Loss: 173.9143
F1-Score: 0.

In [6]:
class GraphModel(nn.Module):
  
  def __init__(self, g, input_size, hidden_size, output_size):
      super().__init__()

      self.g = g
      self.layers = nn.ModuleList()
      self.layers.append(GATConv(input_size, hidden_size, 4))
      self.layers.append(GATConv(4*hidden_size, hidden_size, 4))
      self.layers.append(GATConv(4*hidden_size, output_size, 6))
      
  def forward(self, inputs):
      outputs = inputs
      outputs = self.layers[0](self.g, outputs)
      outputs = F.elu(outputs.flatten(start_dim = 1))
      outputs = self.layers[1](self.g, outputs).flatten(start_dim = 1) + outputs
      outputs = F.elu(outputs)
      outputs = self.layers[2](self.g, outputs)
      outputs = outputs.mean(axis = 1)
      return nn.Sigmoid()(outputs)

NameError: ignored